In [ ]:
!nvidia-smi -L

!nvidia-smi


GPU 0: Tesla T4 (UUID: GPU-ee2f41a8-2e3a-654a-c851-a5760e01f7b7)
Tue Nov 18 09:35:10 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   47C    P8             11W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |       

 # **Install libraries** 🏗️
 This cell will take a little while to download several libraries, including Whisper.

 ---

In [ ]:
! pip install git+https://github.com/openai/whisper.git
! pip install yt-dlp

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-14mm7n4a
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-14mm7n4a
  Resolved https://github.com/openai/whisper.git to commit c0d2f624c09dc18e709e37c2ad90c039a4eb72a2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for openai-whisper: filename=openai_whisper-20250625-py3-none-any.whl size=803979 sha256=fcd9313f9f26b0e04dce1d060f70c45eb2a19febd3da01bc5cda00e307a85d82
  Stored in directory: /tmp/pip-ephem-wheel-cache-8xl_r1ei/wheels/c3/03/25/5e0ba78bc27a3a089f137c9f1d92fdfce16d06996c071a016c
Successfully built openai-whisper
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.0/180.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 56.5 MB/s eta 0:00:00


In [ ]:


import sys
import warnings
import whisper
from pathlib import Path
import yt_dlp
import subprocess
import torch
import shutil
import numpy as np
from IPython.display import display, Markdown, YouTubeVideo

device = torch.device('cuda:0')
print('Using device:', device, file=sys.stderr)

Using device: cuda:0


In [ ]:
# Uncomment to copy generated images to drive, faster than downloading directly from colab in my experience.
from google.colab import drive
drive_mount_path = Path("/") / "content" / "drive"
drive.mount(str(drive_mount_path))
drive_mount_path /= "My Drive"

drive_path = "Colab Notebooks/Whisper Youtube"



drive_whisper_path = drive_mount_path / Path(drive_path.lstrip("/"))
drive_whisper_path.mkdir(parents=True, exist_ok=True)

Mounted at /content/drive


# Open Ai Whisper

these are the models: 'tiny.en', 'tiny', 'base.en', 'base', 'small.en', 'small', 'medium.en', 'medium', 'large'

In [ ]:
Model = 'medium'

In [ ]:
whisper_model = whisper.load_model(Model)

if Model in whisper.available_models():
    display(Markdown(
        f"**{Model} model is selected.**"
    ))
else:
    display(Markdown(
        f"**{Model} model is no longer available.** Please select one of the following: - {' - '.join(whisper.available_models())}"
    ))


100%|█████████████████████████████████████| 1.42G/1.42G [00:16<00:00, 92.5MiB/s]


**medium model is selected.**

In [ ]:
Type = "Youtube video or playlist"

In [ ]:
# add the path of video or playlist
URL = "https://youtu.be/L_Guz73e6fw"

In [ ]:
# Google Drive video, audio (mp4, wav), or folder containing video and/or audio files
video_path = "Colab Notebooks/transcription/my_video.mp4"

In [ ]:
video_path_local_list = []

if Type == "Youtube video or playlist":

    ydl_opts = {
        'format': 'm4a/bestaudio/best',
        'outtmpl': '%(id)s.%(ext)s',
        # ℹ️ See help(yt_dlp.postprocessor) for a list of available Postprocessors and their arguments
        'postprocessors': [{  # Extract audio using ffmpeg
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'wav',
        }]
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        error_code = ydl.download([URL])
        list_video_info = [ydl.extract_info(URL, download=False)]

    for video_info in list_video_info:
        video_path_local_list.append(Path(f"{video_info['id']}.wav"))

elif Type == "Google Drive":
    # video_path_drive = drive_mount_path / Path(video_path.lstrip("/"))
    video_path = drive_mount_path / Path(video_path.lstrip("/"))
    if video_path.is_dir():
        for video_path_drive in video_path.glob("**/*"):
            if video_path_drive.is_file():
                display(Markdown(f"**{str(video_path_drive)} selected for transcription.**"))
            elif video_path_drive.is_dir():
                display(Markdown(f"**Subfolders not supported.**"))
            else:
                display(Markdown(f"**{str(video_path_drive)} does not exist, skipping.**"))
            video_path_local = Path(".").resolve() / (video_path_drive.name)
            shutil.copy(video_path_drive, video_path_local)
            video_path_local_list.append(video_path_local)
    elif video_path.is_file():
        video_path_local = Path(".").resolve() / (video_path.name)
        shutil.copy(video_path, video_path_local)
        video_path_local_list.append(video_path_local)
        display(Markdown(f"**{str(video_path)} selected for transcription.**"))
    else:
        display(Markdown(f"**{str(video_path)} does not exist.**"))

else:
    raise(TypeError("Please select supported input type."))

for video_path_local in video_path_local_list:
    if video_path_local.suffix == ".mp4":
        video_path_local = video_path_local.with_suffix(".wav")
        result  = subprocess.run(["ffmpeg", "-i", str(video_path_local.with_suffix(".mp4")), "-vn", "-acodec", "pcm_s16le", "-ar", "16000", "-ac", "1", str(video_path_local)])



[youtube] Extracting URL: https://youtu.be/L_Guz73e6fw
[youtube] L_Guz73e6fw: Downloading webpage


[youtube] L_Guz73e6fw: Downloading android sdkless player API JSON
[youtube] L_Guz73e6fw: Downloading web safari player API JSON


[youtube] L_Guz73e6fw: Downloading m3u8 information


[info] L_Guz73e6fw: Downloading 1 format(s): 140
[download] Sleeping 5.00 seconds as required by the site...
[download] Destination: L_Guz73e6fw.m4a
[download] 100% of  133.30MiB in 00:00:09 at 13.59MiB/s  
[FixupM4a] Correcting container of "L_Guz73e6fw.m4a"
[ExtractAudio] Destination: L_Guz73e6fw.wav
Deleting original file L_Guz73e6fw.m4a (pass -k to keep)
[youtube] Extracting URL: https://youtu.be/L_Guz73e6fw
[youtube] L_Guz73e6fw: Downloading webpage


[youtube] L_Guz73e6fw: Downloading android sdkless player API JSON
[youtube] L_Guz73e6fw: Downloading web safari player API JSON


[youtube] L_Guz73e6fw: Downloading m3u8 information


['Auto detection', 'Afrikaans', 'Albanian', 'Amharic', 'Arabic', 'Armenian', 'Assamese', 'Azerbaijani', 'Bashkir', 'Basque', 'Belarusian', 'Bengali', 'Bosnian', 'Breton', 'Bulgarian', 'Burmese', 'Castilian', 'Catalan', 'Chinese', 'Croatian', 'Czech', 'Danish', 'Dutch', 'English', 'Estonian', 'Faroese', 'Finnish', 'Flemish', 'French', 'Galician', 'Georgian', 'German', 'Greek', 'Gujarati', 'Haitian', 'Haitian Creole', 'Hausa', 'Hawaiian', 'Hebrew', 'Hindi', 'Hungarian', 'Icelandic', 'Indonesian', 'Italian', 'Japanese', 'Javanese', 'Kannada', 'Kazakh', 'Khmer', 'Korean', 'Lao', 'Latin', 'Latvian', 'Letzeburgesch', 'Lingala', 'Lithuanian', 'Luxembourgish', 'Macedonian', 'Malagasy', 'Malay', 'Malayalam', 'Maltese', 'Maori', 'Marathi', 'Moldavian', 'Moldovan', 'Mongolian', 'Myanmar', 'Nepali', 'Norwegian', 'Nynorsk', 'Occitan', 'Panjabi', 'Pashto', 'Persian', 'Polish', 'Portuguese', 'Punjabi', 'Pushto', 'Romanian', 'Russian', 'Sanskrit', 'Serbian', 'Shona', 'Sindhi', 'Sinhala', 'Sinhalese', 'Slovak', 'Slovenian', 'Somali', 'Spanish', 'Sundanese', 'Swahili', 'Swedish', 'Tagalog', 'Tajik', 'Tamil', 'Tatar', 'Telugu', 'Thai', 'Tibetan', 'Turkish', 'Turkmen', 'Ukrainian', 'Urdu', 'Uzbek', 'Valencian', 'Vietnamese', 'Welsh', 'Yiddish', 'Yoruba']

 # Language spoken in the audio, use `Auto detection` to let Whisper detect the language.

In [ ]:
language = "English"

In [ ]:
verbose = 'Live transcription'

In [ ]:
# ['txt', 'vtt', 'srt', 'tsv', 'json', 'all']

output_format = 'all'

In [ ]:
# ['transcribe', 'translate']

task = 'transcribe'

# Optional task

In [ ]:
temperature = 0.3 # {type:"slider", min:0, max:1, step:0.05}

In [ ]:
# Temperature to increase when falling back when the decoding fails to meet either of the thresholds below.

temperature_increment_on_fallback = 0.2   #{type:"slider", min:0, max:1, step:0.05}

In [ ]:
# Number of candidates when sampling with non-zero temperature.

best_of = 5

In [ ]:
# Number of beams in beam search, only applicable when temperature is zero.
beam_size = 8

In [ ]:
# patience value to use in beam decoding, as in [*Beam Decoding with Controlled Patience*](https://arxiv.org/abs/2204.05424), the default (1.0) is equivalent to conventional beam search.

patience = 1.0

In [ ]:
logprob_threshold = -1.0

In [ ]:
length_penalty = -0.05
suppress_tokens = "-1"
initial_prompt = ""
condition_on_previous_text = True
fp16 = True
compression_ratio_threshold = 2.4
no_speech_threshold = 0.6

In [ ]:
verbose_lut = {
    'Live transcription': True,
    'Progress bar': False,
    'None': None
}

In [ ]:
args = dict(
    language = (None if language == "Auto detection" else language),
    verbose = verbose_lut[verbose],
    task = task,
    temperature = temperature,
    temperature_increment_on_fallback = temperature_increment_on_fallback,
    best_of = best_of,
    beam_size = beam_size,
    patience=patience,
    length_penalty=(length_penalty if length_penalty>=0.0 else None),
    suppress_tokens=suppress_tokens,
    initial_prompt=(None if not initial_prompt else initial_prompt),
    condition_on_previous_text=condition_on_previous_text,
    fp16=fp16,
    compression_ratio_threshold=compression_ratio_threshold,
    logprob_threshold=logprob_threshold,
    no_speech_threshold=no_speech_threshold
)

In [ ]:
temperature = args.pop("temperature")
temperature_increment_on_fallback = args.pop("temperature_increment_on_fallback")
if temperature_increment_on_fallback is not None:
    temperature = tuple(np.arange(temperature, 1.0 + 1e-6, temperature_increment_on_fallback))
else:
    temperature = [temperature]

In [ ]:
if Model.endswith(".en") and args["language"] not in {"en", "English"}:
    warnings.warn(f"{Model} is an English-only model but receipted '{args['language']}'; using English instead.")
    args["language"] = "en"


In [ ]:
for video_path_local in video_path_local_list:
    display(Markdown(f"### {video_path_local}"))

    video_transcription = whisper.transcribe(
        whisper_model,
        str(video_path_local),
        temperature=temperature,
        **args,
    )

    # Save output
    whisper.utils.get_writer(
        output_format=output_format,
        output_dir=video_path_local.parent
    )(
        video_transcription,
        str(video_path_local.stem),
        options=dict(
            highlight_words=False,
            max_line_count=None,
            max_line_width=None,
        )
    )

    def exportTranscriptFile(ext: str):
        local_path = video_path_local.parent / video_path_local.with_suffix(ext).name
        export_path = drive_whisper_path / video_path_local.with_suffix(ext).name
        shutil.copy(
            local_path,
            export_path
        )
        display(Markdown(f"**Transcript file created: {export_path}**"))

    if output_format=="all":
        for ext in ('.txt', '.vtt', '.srt', '.tsv', '.json'):
            exportTranscriptFile(ext)
    else:
        exportTranscriptFile("." + output_format)

### L_Guz73e6fw.wav

[00:00.000 --> 00:04.000]  We have been a misunderstood and badly mocked org for a long time.
[00:04.000 --> 00:06.000]  Like when we started,
[00:07.500 --> 00:10.840]  when we like announced the org at the end of 2015,
[00:10.840 --> 00:12.380]  and said we're going to work on AGI,
[00:12.380 --> 00:14.600]  like people thought we were batshit insane.
[00:14.600 --> 00:15.260]  Yeah.
[00:15.260 --> 00:18.140]  You know, like, I remember at the time,
[00:18.140 --> 00:24.300]  an eminent AI scientist at a large industrial AI lab
[00:24.300 --> 00:27.060]  was like DMing individual reporters,
[00:27.060 --> 00:29.640]  being like, you know, these people aren't very good,
[00:29.640 --> 00:31.480]  and it's ridiculous to talk about AGI,
[00:31.480 --> 00:33.100]  and I can't believe you're giving them time of day.
[00:33.100 --> 00:35.480]  And it's like, that was the level of like,
[00:35.480 --> 00:37.280]  pettiness and rancor in the field
[00:37.280 --> 00:40.400]  that a new group 

**Transcript file created: /content/drive/My Drive/Colab Notebooks/Whisper Youtube/L_Guz73e6fw.txt**

**Transcript file created: /content/drive/My Drive/Colab Notebooks/Whisper Youtube/L_Guz73e6fw.vtt**

**Transcript file created: /content/drive/My Drive/Colab Notebooks/Whisper Youtube/L_Guz73e6fw.srt**

**Transcript file created: /content/drive/My Drive/Colab Notebooks/Whisper Youtube/L_Guz73e6fw.tsv**

**Transcript file created: /content/drive/My Drive/Colab Notebooks/Whisper Youtube/L_Guz73e6fw.json**